In [1]:
import pandas as pd

In [2]:
from selenium import webdriver

In [3]:
from selenium.webdriver.common.keys import Keys

In [297]:
df = pd.DataFrame(columns=['id','message','date'])
df.head()

,id,message,date


In [4]:
chromedriver = '/usr/local/bin/chromedriver'

In [5]:
driver = webdriver.Chrome(chromedriver)

select = driver.find_elements_by_css_selector
select_one = driver.find_element_by_css_selector


In [266]:
driver.get('https://www.linkedin.com/')

In [258]:
#if on the main linked in with the sign in on top right
try:
    sign_in = select_one('a.nav__button-secondary')
    sign_in.click()
    input_name = select_one('input#username')
    input_name.send_keys('keri@multerer.com')
    input_pwd = select_one('input#password')
    input_pwd.send_keys('jfdkfjadluivjdkk5')
    login_action = select_one('div.login__form_action_container ')
    login_action.click()
except:
    email_input = select_one('input.login-email')
    email_input.click()
    email_input.send_keys('keri@multerer.com')
    password_input = select_one('input.login-password')
    password_input.click()
    password_input.send_keys('jfdkfjadluivjdkk5')
    submit = select_one('#login-submit')
    submit.click()
except:
    driver.get(sign_in.get_attribute('href'))
    input_name = select_one('input#username')
    input_name.click()
    input_name.send_keys('keri@multerer.com')
    input_password = select_one('input#password')
    input_password.click()
    input_password.send_keys('jfdkfjadluivjdkk5')
    login_btn = select_one('button.btn__primary--large.from__button--floating')
    login_btn.click()
except:
    print('nope, didnt work')


SyntaxError: default 'except:' must be last (<ipython-input-258-887471ffc0dd>, line 10)

# Getting list of conversations on the left and then clicking through them to capture the full message thread

In [285]:
#accessing the messages once logged in
message_btn = select_one('span#messaging-tab-icon')
message_btn.click()

In [286]:
convo_list = select('li.msg-conversation-listitem')

In [298]:
for convo in convo_list:
    convo.click()
    try:
        message = select_one('div.msg-s-message-list').text
        print(message)
        message_list = message.split('\n')
        print(message_list)
        
        if message_list[0] == 'Recruiter':
            df = df.append({'id': message_list[0], 'message': message, 'date': message_list[1]}, ignore_index=True)
        elif '2' in message_list[0]:
            df = df.append({'id': message_list[3], 'message': message, 'date': message_list[0]}, ignore_index=True)
        else:
            df = df.append({'id': message_list[0], 'message': message, 'date': message_list[5]}, ignore_index=True)

        print(message_list)
        #name = message_list[4]
        #date = message_list[1]
        #df = df.append({'id' : name , 'message' : message, 'date': date} , ignore_index=True)
    except:
        df = df.append({'id': '' , 'message' : 'advertisement', 'date': ''}, ignore_index=True)

Faisal Zahid
Status is reachable
Faisal Zahid
· 1st
Technologist
JAN 28
Faisal Zahid sent the following message at 2:37 PM
View Faisal’s profile
Faisal Zahid
Faisal Zahid 2:37 PM
Hi Keri, We briefly met at EHF welcome week when Boyd introduced me regarding my game using XNA earlier. I recently came across your skin care recommendation model. Looks very interesting and we've been working on similar stuff (more generic) at https://whydis.com. Would love to chat more on this.
Whydis?
whydis.com
Faisal Zahid is now a connection.
JAN 30
View Keri’s profile
Keri Multerer
Keri Multerer 12:43 PM
Hi Faisal, Thanks for reaching out. Hope you're enjoying Wellington. I would love to chat more about the skin recommendation app and your app. Looks very interesting! Let me know where you would like to meet. I'm pretty open now that my kids are back in school.

You can reach also me at keri@multerer.com. 

Cheers,
Keri
View Faisal’s profile
Faisal Zahid
Faisal Zahid 3:14 PM
Keri, we are in Palmerston 

Recruiter
Feb 22
Andrew Yin sent the following message at 5:36 PM
View Andrew’s profile
Andrew Yin
Andrew Yin
EY-Parthenon Seeking Your Perspective on Software Programs
Dear Keri, 

I hope this message finds you well. My name is Andrew Yin and I’m an associate at EY-Parthenon, a market research and consulting firm. My team is looking to share with you a brief ~10 minute survey to learn about your perspectives on bootcamps, as it looks like you recently graduated from a software bootcamp (e.g. coding, UI/UX, data analytics, cyber security, etc.). We are happy to compensate you with a $30 Tango gift card (can be redeemed at Amazon, Target, and many other retailers, www.tangocard.com). Due to Linkedin policy, we are unable to share the survey through Linkedin, but would be happy to share it over email. If you are interested, please respond with your email address and we will send the survey shortly thereafter.

For context, my team is currently conducting research on software bootcamps th

Elena Poughia
Status is reachable
Elena Poughia
· 1st
MD at Dataconomy Media and Data Natives | building deep tech communities
SEP 28, 2018
Elena Poughia sent the following message at 1:02 AM
View Elena’s profile
Elena Poughia
Elena Poughia 1:02 AM
Hi Keri,
It would be great to meet you tonight at our Big Data Event which will focus on healthcare. Here is the link to register: https://www.eventbrite.co.uk/e/big-data-healthcare-seattle-tickets-49666885993. 

Let me know if you are interested and available.

Best, Elena.
Big Data Healthcare, Seattle
eventbrite.co.uk
SEP 29, 2018
Elena Poughia is now a connection.
['Elena Poughia', 'Status is reachable', 'Elena Poughia', '· 1st', 'MD at Dataconomy Media and Data Natives | building deep tech communities', 'SEP 28, 2018', 'Elena Poughia sent the following message at 1:02 AM', 'View Elena’s profile', 'Elena Poughia', 'Elena Poughia 1:02 AM', 'Hi Keri,', 'It would be great to meet you tonight at our Big Data Event which will focus on healthca

Daniel Jang
Daniel Jang
· 1st
Financial Consultant at AXA Advisors
JUL 20, 2018
Daniel Jang sent the following message at 8:21 AM
View Daniel’s profile
Daniel Jang
Daniel Jang 8:21 AM
Hey, we both attended the Microsoft event, but did not get a chance to meet. Figured we should stay connected!
JUL 25, 2018
Daniel Jang is now a connection.
['Daniel Jang', 'Daniel Jang', '· 1st', 'Financial Consultant at AXA Advisors', 'JUL 20, 2018', 'Daniel Jang sent the following message at 8:21 AM', 'View Daniel’s profile', 'Daniel Jang', 'Daniel Jang 8:21 AM', 'Hey, we both attended the Microsoft event, but did not get a chance to meet. Figured we should stay connected!', 'JUL 25, 2018', 'Daniel Jang is now a connection.']
['Daniel Jang', 'Daniel Jang', '· 1st', 'Financial Consultant at AXA Advisors', 'JUL 20, 2018', 'Daniel Jang sent the following message at 8:21 AM', 'View Daniel’s profile', 'Daniel Jang', 'Daniel Jang 8:21 AM', 'Hey, we both attended the Microsoft event, but did not get a chance 

Recruiter
May 24, 2018
LinkedIn Member sent the following message at 7:10 AM
View LinkedIn Member’s profile
LinkedIn Member
LinkedIn Member
Data Scientist eCommerce Platform-Fortune 500 company-Bellevue, WA
Dear Keri, 

Hi,

Hope you are doing great! we have long term open end position with our client in Bellevue, WA. Attached is job description for you if interested please share resume/ contact details and best time to reach out to you.
 
Role Purpose:
The SCM team within eCommerce Strategy and Operations division manages the forecasting and supply chain management functions across all SEA eCommerce product categories. They are also responsible for collaborating with the eCommerce Business Management team, the Samsung SCM team, and the product marketing teams on a weekly basis for ensuring product availability to meet forecasted demand.

common Essential Duties & Resp.
• Discover and apply insights from massive data sets of structured, semi-structured, and unstructured data
• Collabor

MAR 17, 2018
Keri Multerer sent the following message at 6:19 PM
View Keri’s profile
Keri Multerer
Keri Multerer 6:19 PM
clinical data analyst
Hi Mariah,
I am contacting you in regards to opportunities at Seattle Children's that involve machine learning, working with large datasets, and analyzing data. I have just completed an extensive data science course at Galvanize and have a Master of Science in genetics. I have stayed home with my children for a while and would love to apply my skills to helping children. I also appreciate any feedback you have as I am just beginning my search for opportunities.

Best,
Keri Multerer
['MAR 17, 2018', 'Keri Multerer sent the following message at 6:19 PM', 'View Keri’s profile', 'Keri Multerer', 'Keri Multerer 6:19 PM', 'clinical data analyst', 'Hi Mariah,', "I am contacting you in regards to opportunities at Seattle Children's that involve machine learning, working with large datasets, and analyzing data. I have just completed an extensive data sci

Ha Tran
Status is reachable
Ha Tran
· 1st
Data Scientist
FEB 10, 2018
Ha Tran sent the following message at 8:31 AM
View Ha’s profile
Ha Tran
Ha Tran 8:31 AM
Thanks for endorsing me for Poisson Analysis!
Ha Tran
['Ha Tran', 'Status is reachable', 'Ha Tran', '· 1st', 'Data Scientist', 'FEB 10, 2018', 'Ha Tran sent the following message at 8:31 AM', 'View Ha’s profile', 'Ha Tran', 'Ha Tran 8:31 AM', 'Thanks for endorsing me for Poisson Analysis!', 'Ha Tran']
['Ha Tran', 'Status is reachable', 'Ha Tran', '· 1st', 'Data Scientist', 'FEB 10, 2018', 'Ha Tran sent the following message at 8:31 AM', 'View Ha’s profile', 'Ha Tran', 'Ha Tran 8:31 AM', 'Thanks for endorsing me for Poisson Analysis!', 'Ha Tran']
Brett Smith
Brett Smith
· 1st
Software Engineer - Research at Arivale
DEC 11, 2017
Brett Smith sent the following message at 11:08 AM
View Brett’s profile
Brett Smith
Brett Smith 11:08 AM
Brett from Galvanize DSI here!
DEC 12, 2017
Brett Smith is now a connection.
['Brett Smith', 'Brett Sm

Simon Robinson
Simon Robinson
· 1st
Chief IP Strategy and Co-Founder at IntuitiveX
JAN 14, 2017
Keri Multerer sent the following message at 6:25 AM
View Keri’s profile
Keri Multerer
Keri Multerer 6:25 AM
Hi Simon,
Jody Stoehr forwarded your information to me. I am getting back into the workforce (field of science) after taking a break to raise kids. I welcome opinions on big changes in the last 10 years. I'm very interested in pursuing data science and teaching myself python. 
- Keri Multerer
Simon Robinson is now a connection.
['Simon Robinson', 'Simon Robinson', '· 1st', 'Chief IP Strategy and Co-Founder at IntuitiveX', 'JAN 14, 2017', 'Keri Multerer sent the following message at 6:25 AM', 'View Keri’s profile', 'Keri Multerer', 'Keri Multerer 6:25 AM', 'Hi Simon,', "Jody Stoehr forwarded your information to me. I am getting back into the workforce (field of science) after taking a break to raise kids. I welcome opinions on big changes in the last 10 years. I'm very interested in pur

Julie Notarianni
Status is reachable
Julie Notarianni
· 1st
Creative Strategist
MAR 29, 2016
Julie Notarianni sent the following message at 2:49 PM
View Julie’s profile
Julie Notarianni
Julie Notarianni 2:49 PM
Congrats on the anniversary! Hope you're doing well.
MAR 31, 2016
Keri Multerer sent the following message at 10:57 AM
View Keri’s profile
Keri Multerer
Keri Multerer 10:57 AM
Thanks Julie! Hope you're doing well also.
JUL 29, 2016
View Julie’s profile
Julie Notarianni
Julie Notarianni 2:40 AM
Congrats on the anniversary! Hope you're doing well.
['Julie Notarianni', 'Status is reachable', 'Julie Notarianni', '· 1st', 'Creative Strategist', 'MAR 29, 2016', 'Julie Notarianni sent the following message at 2:49 PM', 'View Julie’s profile', 'Julie Notarianni', 'Julie Notarianni 2:49 PM', "Congrats on the anniversary! Hope you're doing well.", 'MAR 31, 2016', 'Keri Multerer sent the following message at 10:57 AM', 'View Keri’s profile', 'Keri Multerer', 'Keri Multerer 10:57 AM', "Than

In [299]:
df.head(20)

,id,message,date
0,Faisal Zahid,Faisal Zahid\nStatus is reachable\nFaisal Zahi...,JAN 28
1,,advertisement,
2,Recruiter,Recruiter\nFeb 22\nAndrew Yin sent the followi...,Feb 22
3,Yiran Sherry,Yiran Sherry\nStatus is reachable\nYiran Sherr...,FEB 24
4,,advertisement,
5,,advertisement,
6,Recruiter,"Recruiter\nOct 17, 2018\nSophia Georgantonis s...","Oct 17, 2018"
7,Elena Poughia,Elena Poughia\nStatus is reachable\nElena Poug...,"SEP 28, 2018"
8,,advertisement,
9,Charlie Redding,"AUG 11, 2018\nCharlie Redding sent the followi...","AUG 11, 2018"


# Getting the stats for visits

In [193]:
#go to profile page
profile = select_one('li#profile-nav-item')
profile.click()
dropdown = select_one('ul#nav-settings__dropdown-options')
url = dropdown.find_element_by_css_selector('a')
url.get_attribute('href')
driver.get(url.get_attribute('href'))

In [194]:
#get the profile view information
views = select_one('a.profile-views')
views.get_attribute('href')
driver.get(views.get_attribute('href'))

In [195]:
#number of views in the last 90 days
views = select_one('span.me-wvmp-views__90-days-views')
views.text

'17'

In [196]:
#access all views page
all_views = select_one('li.ember-view a')
driver.get(all_views.get_attribute('href'))

In [202]:
#accessing your social selling index: Must be logged in then go to https://www.linkedin.com/sales/ssi
driver.get('https://www.linkedin.com/sales/ssi')

In [222]:
body = select_one('article#main')

In [208]:
wrappers = body.find_element_by_css_selector('div#current-barchart-wrapper')

In [209]:
wrappers.text

'0\n0\n0\n0\nEstablish your professional brand\nFind the right people\nEngage with insights\nBuild relationships\n25\n25\n25\n25\n9.97\n5.32\n0.6\n3.07'

In [210]:
score_list = wrappers.text.split('\n')

In [211]:
score_list

['0',
 '0',
 '0',
 '0',
 'Establish your professional brand',
 'Find the right people',
 'Engage with insights',
 'Build relationships',
 '25',
 '25',
 '25',
 '25',
 '9.97',
 '5.32',
 '0.6',
 '3.07']

In [213]:
labels = score_list[4:8]

In [214]:
scores = score_list[-4:]

In [215]:
print(labels)
print(scores)

['Establish your professional brand', 'Find the right people', 'Engage with insights', 'Build relationships']
['9.97', '5.32', '0.6', '3.07']


In [217]:
list(zip(labels,scores))

[('Establish your professional brand', '9.97'),
 ('Find the right people', '5.32'),
 ('Engage with insights', '0.6'),
 ('Build relationships', '3.07')]

In [248]:
weekly_totals = select_one('table')

In [253]:
totals = weekly_totals.find_elements_by_css_selector('td ')

In [255]:
len(totals)

8

In [256]:
for total in totals:
    print(total.text)

In [56]:
#this was on the page when linked in asked if my info was correct
info_correct = select_one('button.primary-action')
info_correct.click()

In [ ]:
message = select_one('p.msg-spinmail-thread__message-body t-14 t-black t-normal msg-spinmail-thread__message-body--with-spacing')